In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib as mpl
import copy
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

In [2]:
disease_genes_read = "/home/androx/Documents/trabalho/datasets/Disease_genes/genehancer/GeneHancer_v5.20_processed.gff"
disease_genes_write =  "/home/androx/Documents/trabalho/datasets/Disease_genes/genehancer/GeneHancer_v5.20_complete.gff"
dg_df = pd.read_csv(disease_genes_read, sep="\t")

dg_df

,#chrom,source,feature name,start,end,score,strand,frame,attributes,size,cytoband
0,chr20,GeneHancer,Enhancer,208887,209430,0.76,.,.,genehancer_id=GH20J000209;connected_gene=TBC1D...,543,p13
1,chr20,GeneHancer,Promoter/Enhancer,226800,228800,0.71,.,.,genehancer_id=GH20J000227;connected_gene=DEFB1...,2000,p13
2,chr20,GeneHancer,Promoter/Enhancer,266199,267649,1.72,.,.,genehancer_id=GH20J000266;connected_gene=ENSG0...,1450,p13
3,chr20,GeneHancer,Promoter/Enhancer,279139,279609,0.51,.,.,genehancer_id=GH20J000280;connected_gene=C20or...,470,p13
4,chr20,GeneHancer,Promoter/Enhancer,289399,292542,2.12,.,.,genehancer_id=GH20J000289;connected_gene=C20or...,3143,p13
...,...,...,...,...,...,...,...,...,...,...,...
419015,chr19,GeneHancer,Enhancer,49645261,49646700,0.14,.,.,genehancer_id=GH19J049645;connected_gene=HSALN...,1439,q13.33
419016,chr17,GeneHancer,Enhancer,17222391,17224411,0.26,.,.,genehancer_id=GH17J017222;connected_gene=FLCN;...,2020,p11.2
419017,chr17,GeneHancer,Enhancer,17220946,17221921,0.14,.,.,genehancer_id=GH17J017220;connected_gene=NT5M;...,975,p11.2
419018,chr17,GeneHancer,Enhancer,17215651,17216481,0.26,.,.,genehancer_id=GH17J017216;connected_gene=FLCN;...,830,p11.2


In [3]:
genes_list = []
for row in dg_df["attributes"]:
    row_elements = row.split(";")
    for i in range (1, len(row_elements)-1, 2):
        gene_symbol = row_elements[i].split("=")[1]
        genes_list.append(gene_symbol)

len(genes_list)

2552245

In [4]:
def writeNewtxt(file_path):
    """Writes a pandas dataframe into txt, columns separated by tabs ("\t")
    """
    f = open(file_path, 'w')
    string = ""
    for gene_symbol in genes_list:
        string += " " + gene_symbol
    f.write(string)
    f.close

In [5]:
gene_symbols_path = "/home/androx/Documents/trabalho/datasets/Disease_genes/genehancer/DGsymbols.txt"
writeNewtxt(gene_symbols_path)

In [6]:
idConvert_read= "/home/androx/Documents/trabalho/datasets/Disease_genes/genehancer/gProfiler_hsapiens_01-07-2024_17-15-03.csv"
idConvert_df = pd.read_csv(idConvert_read, sep=",")

idConvert_df

,initial_alias,converted_alias,name,description,namespace
0,TBC1D20,ENSG00000125875,TBC1D20,TBC1 domain family member 20 [Source:HGNC Symb...,"ENTREZGENE,GENECARDS,HGNC,UNIPROT_GN,WIKIGENE"
1,PIR-34165,NaN,NaN,NaN,NaN
2,DEFB128,ENSG00000185982,DEFB128,defensin beta 128 [Source:HGNC Symbol;Acc:HGNC...,"ENTREZGENE,GENECARDS,HGNC,UNIPROT_GN,WIKIGENE"
3,DEFB129,ENSG00000125903,DEFB129,defensin beta 129 [Source:HGNC Symbol;Acc:HGNC...,"ENTREZGENE,GENECARDS,HGNC,UNIPROT_GN,WIKIGENE"
4,HSALNG0128009,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2573611,HSALNG0131740,NaN,NaN,NaN,NaN
2573612,PIR-39425,NaN,NaN,NaN,NaN
2573613,HSALNG0131742,NaN,NaN,NaN,NaN
2573614,HSALNG0131741,NaN,NaN,NaN,NaN


In [7]:
def create_newCol():
    ENG_ids = []
    converted_line_n = 0
    for row in dg_df["attributes"]:
        ENG_ids_row = ""
        for i in range (row.count("gene=")):
            dg = idConvert_df["converted_alias"][converted_line_n]
            ENG_ids_row += str(dg) + ", "
            converted_line_n += 1
        ENG_ids.append(str(ENG_ids_row[:-2]))
    return ENG_ids
    

In [8]:
def writeNewtxt(file_path, df):
    """Writes a pandas dataframe into txt, columns separated by tabs ("\t")
    """
    f = open(file_path, 'w')
    str_cols = ""
    for col in df.columns[:-1]:
        str_cols += col + "\t"
    str_cols += df.columns.values[-1]+ "\n"
    f.write(str(str_cols))
    for row in df.values:
        str_row = ""
        for col in row[:-1]:
            str_row += str(col) + "\t"
        str_row += str(row[-1])+ "\n"
        f.write(str_row)
    f.close

In [9]:
gene_ENGids = create_newCol()

In [10]:
dg_df.insert(len(dg_df.columns), "connected_genes_ENGids", gene_ENGids, True)

In [11]:
dg_df.drop(columns=['connected_genes_ENGids'])

,#chrom,source,feature name,start,end,score,strand,frame,attributes,size,cytoband
0,chr20,GeneHancer,Enhancer,208887,209430,0.76,.,.,genehancer_id=GH20J000209;connected_gene=TBC1D...,543,p13
1,chr20,GeneHancer,Promoter/Enhancer,226800,228800,0.71,.,.,genehancer_id=GH20J000227;connected_gene=DEFB1...,2000,p13
2,chr20,GeneHancer,Promoter/Enhancer,266199,267649,1.72,.,.,genehancer_id=GH20J000266;connected_gene=ENSG0...,1450,p13
3,chr20,GeneHancer,Promoter/Enhancer,279139,279609,0.51,.,.,genehancer_id=GH20J000280;connected_gene=C20or...,470,p13
4,chr20,GeneHancer,Promoter/Enhancer,289399,292542,2.12,.,.,genehancer_id=GH20J000289;connected_gene=C20or...,3143,p13
...,...,...,...,...,...,...,...,...,...,...,...
419015,chr19,GeneHancer,Enhancer,49645261,49646700,0.14,.,.,genehancer_id=GH19J049645;connected_gene=HSALN...,1439,q13.33
419016,chr17,GeneHancer,Enhancer,17222391,17224411,0.26,.,.,genehancer_id=GH17J017222;connected_gene=FLCN;...,2020,p11.2
419017,chr17,GeneHancer,Enhancer,17220946,17221921,0.14,.,.,genehancer_id=GH17J017220;connected_gene=NT5M;...,975,p11.2
419018,chr17,GeneHancer,Enhancer,17215651,17216481,0.26,.,.,genehancer_id=GH17J017216;connected_gene=FLCN;...,830,p11.2


In [12]:
dg_df

,#chrom,source,feature name,start,end,score,strand,frame,attributes,size,cytoband,connected_genes_ENGids
0,chr20,GeneHancer,Enhancer,208887,209430,0.76,.,.,genehancer_id=GH20J000209;connected_gene=TBC1D...,543,p13,"ENSG00000125875, nan, ENSG00000185982, ENSG000..."
1,chr20,GeneHancer,Promoter/Enhancer,226800,228800,0.71,.,.,genehancer_id=GH20J000227;connected_gene=DEFB1...,2000,p13,"ENSG00000125903, nan, ENSG00000186458, ENSG000..."
2,chr20,GeneHancer,Promoter/Enhancer,266199,267649,1.72,.,.,genehancer_id=GH20J000266;connected_gene=ENSG0...,1450,p13,"ENSG00000272874, nan, ENSG00000247315, ENSG000..."
3,chr20,GeneHancer,Promoter/Enhancer,279139,279609,0.51,.,.,genehancer_id=GH20J000280;connected_gene=C20or...,470,p13,"ENSG00000196476, nan, ENSG00000186458"
4,chr20,GeneHancer,Promoter/Enhancer,289399,292542,2.12,.,.,genehancer_id=GH20J000289;connected_gene=C20or...,3143,p13,"ENSG00000196476, ENSG00000215388, ENSG00000125..."
...,...,...,...,...,...,...,...,...,...,...,...,...
419015,chr19,GeneHancer,Enhancer,49645261,49646700,0.14,.,.,genehancer_id=GH19J049645;connected_gene=HSALN...,1439,q13.33,"ENSG00000170185, ENSG00000250326, ENSG00000293..."
419016,chr17,GeneHancer,Enhancer,17222391,17224411,0.26,.,.,genehancer_id=GH17J017222;connected_gene=FLCN;...,2020,p11.2,"ENSG00000153147, nan, nan, ENSG00000184144, EN..."
419017,chr17,GeneHancer,Enhancer,17220946,17221921,0.14,.,.,genehancer_id=GH17J017220;connected_gene=NT5M;...,975,p11.2,"ENSG00000276691, nan, ENSG00000061273, ENSG000..."
419018,chr17,GeneHancer,Enhancer,17215651,17216481,0.26,.,.,genehancer_id=GH17J017216;connected_gene=FLCN;...,830,p11.2,"ENSG00000268069, ENSG00000079337, ENSG00000229..."


In [13]:
writeNewtxt(disease_genes_write, dg_df)